## Constrained Markov Decision Processes
* このページでは、Eitan Altman先生が執筆された[Constrained Markov Decision Processes](https://www.taylorfrancis.com/books/mono/10.1201/9781315140223/constrained-markov-decision-processes-eitan-altman "Constrained Markov Decision Processes")の理論本で学んだことを書いていこうとおもいます。ここでは第3章について、学んだことをまとめます。




### Occupancy measureとPrimal LP


* Occupancy measureとは、ある方策および初期状態において、各状態行動ペアがどれだけその方策によって訪れられたかを示す指標です。

* 初期状態分布$\beta$、方策 $u$、状態行動ペア $(x, a)$ ,$X_t$ と $A_t$ は時刻 $t$ における状態と行動、$P_{\beta}^{u}$ は初期分布 $\beta$ と方策 $u$ のもとでの確率測度に対して、Occupancy measure $f_{\alpha}(\beta, u; x, a)$ は次のように定義されます。

$$
f_{\alpha}(\beta, u; x, a) \stackrel{\text { def }}{=}(1-\alpha) \sum_{t=1}^{\infty} \alpha^{t-1} P_{\beta}^{u}\left(X_{t}=x, A_{x}=a\right), \quad x \in \mathbf{X}, a \in \mathrm{~A}(x)
$$

In [15]:
import numpy as np

alpha = 0.1
S =2
A =2
transition = np.zeros((S,A,S))
transition[0,0,0] = 0.9
transition[0,0,1] = 0.1
transition[0,1,0] = 0.1
transition[0,1,1] = 0.9
transition[1,0,0] = 0.9
transition[1,0,1] = 0.1
transition[1,1,0] = 0.1
transition[1,1,1] = 0.9
#random s,a trajectory
s = np.random.randint(0,S)
a = np.random.randint(0,A)
trajectory = [] 
for i in range(100):
    trajectory.append((s,a))
    s = np.random.choice(S,p=transition[s,a,:])
    a = np.random.randint(0,A)

def occupancy_measure(trajectory,s,a,alpha):
    sum = 0
    for i in range(len(trajectory)):
        sum += alpha**(len(trajectory)-i)* (trajectory[i][0] == s and trajectory[i][1] == a)
    return sum

occupancy_measure(trajectory,1,0,alpha)

0.10000010010101

### Occupancy measureからPrimal LPへ
* Occupancy measureを用いてコスト関数を表現することで、のちの線形計画問題を解くことができます。この説ではOccupancy measureをなぜ用いるのか？もちいたことでなにがうれしいのかを説明した定理を説明していきます。

#### 完全性(定理3.1)
* 完全性とは、任意のすべての何かに対して、その部分集合のみからそのすべてを表現できるといったニュアンスです。具体的に行きましょう。任意の方策のクラス $\bar{U}$ に対して、$\mathbf{L}_{\bar{U}}^{\alpha}(\beta)=\bigcup_{u \in \bar{U}} f_{\alpha}(\beta, u)$ と定義します。任意の方策サブクラスに対応するOccupancy measureの集合ですね。

* 方策クラス$\bar{U}$が、割引MDPに対して完全であるとは、$\mathbf{L}^{\alpha}(\beta)=\mathbf{L} \frac{\alpha}{U}(\beta)$ が成立するということです。左辺はすべての方策クラスに対応するOccupancy measureであり、右辺は方策サブクラスに対応するOccupancy measureです。等式でつながっていますが、集合間の等式ですので、要素から要素数すべて等しいという意味になります。$\bar{U}$に属する方策のみですべての方策にたいするOccupancy measureを表現できるというわけです。これは次の定理につながります。

**定理3.1(定常方策の集合は完全)**

任意の方策 $u$ に対して、同じ$f_{\alpha}(\beta, u)$ を生成する定常方策 $w$ が存在する。

* 定常方策クラスに対応したOccupancy measureは任意の方策クラスと同等であるわけです。つまり定常方策についてOccupancy measureに関連した問題を解くことは任意の方策クラスについて考えることと同じというわけです。

**TODO proof**

#### Occupancy measureの性質
**定理3.2(Occupancy measureの性質)**

* $\mathbf{L}_{U_{S}}^{\alpha}(\beta)$ は **閉じたポリトープ**である。
* これは、定常方策に対応するOccupancy measureの集合が、決定論的定常方策と対応するoccupancy measureの集合の凸閉包と一致していることを示しています。凸閉包は、コンパクト集合なので凸閉包に属する関数は必ず最大値と最小値を持ちます。Occupancy measureはコスト関数を表現するので、その集合の凸閉包が存在するということは、コスト関数の最小値を解析的に求めることができるということです。

* $\mathbf{L}^{\alpha}(\beta)=\mathbf{L}_{U_{S}}^{\alpha}(\beta)=\overline{c o} \mathbf{L}_{U_{D}}^{\alpha}(\beta)=\mathbf{Q}^{\alpha}(\beta)$ 
* **TODO**

#### 制約付きMDPとLP問題の対応
* 制約付きMDPにおける最適値はLP問題での解とおなじになります。それを以下の定理で示していきます。制約付きMDPは以下のようなLP問題として表現することができます。

* 目的と目的関数：、$\mathcal{C}(\rho):=\langle\rho, c\rangle$の最小値$\mathcal{C}^{*}$を見つける。
* 制約条件：$
\mathcal{D}^{k}(\rho):=\left\langle\rho, d^{k}\right\rangle \leq V_{k}, k=1, \ldots, K, \quad \rho \in \mathbf{Q}^{\alpha}(\beta)
$


**定理3.3**
* $\mathcal{C}^{*}=C_{\alpha}(\beta)$が成立する。つまり、LPの最適値$\mathcal{C}^{*}$は、制約付きMDPの最適値と等しいということを示しています。
* 任意の方策 $u$ に対して、そのOccupancy measure $\rho(u) := f_{\alpha}(\beta, u)$ はLPの実行可能解である。つまり、ある方策に対するOccupancy measureを出すと、それがLP問題における解のひとつになるということです。
* LPは、制約付きMDPが実行可能な場合にのみ実行可能である。LPの最適解$\rho^*$が存在し、それに対応した定常方策$w(\rho^{*})$ はCOPに対して最適となる。  


### ラグランジュを用いた手法
* ラグランジュを使って制約付き問題を解くと嬉しいこととして、制約付き問題を制約なし最適化問題に変換することできることが挙げられます。
* 定理3.6では、ラグランジュをつかった変換後の最適化問題での解が、変換前の制約付き問題での解と等しいということを示しています。
**定理3.6**

1. 最適値関数は、以下を満たす。

$$
C_{\alpha}(\beta)=\inf _{u \in U} \sup _{\lambda \geq 0} J_{\alpha}^{\lambda}(\beta, u)=\inf _{u \in U_{M}} \sup _{\lambda \geq 0} J_{\alpha}^{\lambda}(\beta, u)
$$

ここで、

$$
\begin{align*}
J_{\alpha}^{\lambda}(\beta, u) & :=C_{\alpha}(\beta, u)+\left\langle\lambda, D_{\alpha}(\beta, u)-V\right\rangle \\
& =\sum_{t=1}^{\infty} \alpha^{t-1} E_{\beta}^{u} j^{\lambda}\left(X_{t}, A_{t}\right)-\langle\lambda, V\rangle \\
j^{\lambda}(x, a) & :=c(x, a)+\langle\lambda, d(x, a)\rangle
\end{align*}
$$

2. 方策 $u^{*}$ が制約付きMDPに対して最適であるための必要十分条件は、$C_{\alpha}(\beta)=\sup _{\lambda \geq 0} J_{\alpha}^{\lambda}\left(\beta, u^{*}\right)$ が成り立つことである。
3. 最適値関数は、以下を満たす。

$$
C_{\alpha}(\beta)=\sup _{\lambda \geq 0} \min _{u \in U} J_{\alpha}^{\lambda}(\beta, u)=\sup _{\lambda \geq 0} \min _{u \in U_{D}} J_{\alpha}^{\lambda}(\beta, u)
$$

さらに、以下のような $u^{*} \in U_{S}$ が存在する。

$$
C_{\alpha}(\beta)=\inf _{u \in U} \sup _{\lambda \geq 0} J_{\alpha}^{\lambda}(\beta, u)=\inf _{u \in U_{S}} \sup _{\lambda \geq 0} J_{\alpha}^{\lambda}(\beta, u)=\sup _{\lambda \geq 0} J_{\alpha}^{\lambda}\left(\beta, u^{*}\right)
$$

また、$u^{*}$ は制約付きMDPに対して最適である。

#### 双対問題
* 次のようにラグランジュは以下のように双対問題に変換できます。さらに定理3.7は制約付きMDPの最適値が双対に変化させた問題の解と一致することを示しています。



